# Quadratic Optimization using Simplex computation
### Simplex works only with linear targets, but in case of the objective function, it can be either linear or quadratic

### Our objective here is minimize risk, while targeting a specific return

In [194]:
import pandas as pd
import numpy as np

In [195]:
file = "All_Stocks.xlsx"

In [196]:
data = pd.read_excel(file)

In [197]:
data.head()

,Date,GOOG,AAPL,SBUX,TSLA
0,2019-04-01,1188.479980,49.138439,74.778992,47.737999
1,2019-05-01,1103.630005,42.869724,73.219505,37.032001
2,2019-06-01,1080.910034,48.651680,81.073700,44.692001
3,2019-07-01,1216.680054,52.368397,91.576622,48.321999
4,2019-08-01,1188.099976,51.311405,93.385139,45.122002


In [198]:
data["Date"] = pd.to_datetime(data["Date"], format="%d-%b-%y")

In [199]:
data.head()

,Date,GOOG,AAPL,SBUX,TSLA
0,2019-04-01,1188.479980,49.138439,74.778992,47.737999
1,2019-05-01,1103.630005,42.869724,73.219505,37.032001
2,2019-06-01,1080.910034,48.651680,81.073700,44.692001
3,2019-07-01,1216.680054,52.368397,91.576622,48.321999
4,2019-08-01,1188.099976,51.311405,93.385139,45.122002


In [200]:
data.dtypes

Date    datetime64[ns]
GOOG           float64
AAPL           float64
SBUX           float64
TSLA           float64
dtype: object

Now we must calculate returns from base/start price

We only want to do that for int or float valued columns


In [201]:
returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ["int64", "float64"]]].pct_change()

In [202]:
returns.head()

,GOOG,AAPL,SBUX,TSLA
0,NaN,NaN,NaN,NaN
1,-0.071394,-0.127573,-0.020855,-0.224266
2,-0.020587,0.134873,0.107269,0.206848
3,0.125607,0.076394,0.129548,0.081223
4,-0.023490,-0.020184,0.019749,-0.066222


In [203]:
returns = returns[1:]

In [204]:
stock_names = ["GOOG", "AAPL", "SBUX", "TSLA"]
stock_names

['GOOG', 'AAPL', 'SBUX', 'TSLA']

In [205]:
stock_returns = returns[stock_names]

In [206]:
stock_returns.head()

,GOOG,AAPL,SBUX,TSLA
1,-0.071394,-0.127573,-0.020855,-0.224266
2,-0.020587,0.134873,0.107269,0.206848
3,0.125607,0.076394,0.129548,0.081223
4,-0.023490,-0.020184,0.019749,-0.066222
5,0.026008,0.077038,-0.080829,0.067639


In [207]:
n = len(stock_names)

In [208]:
expected_returns = np.mean(stock_returns)
expected_returns = matrix(expected_returns)
expected_returns

<4x1 matrix, tc='d'>

In [209]:
!pip install CVXOPT

In [210]:
import cvxopt as opt

In [211]:
from cvxopt import matrix, solvers

In [212]:
cov_stocks = stock_returns.cov()
cov_stocks

,GOOG,AAPL,SBUX,TSLA
GOOG,0.005480,0.003654,0.003736,0.010104
AAPL,0.003654,0.009947,0.003483,0.021065
SBUX,0.003736,0.003483,0.006982,0.008112
TSLA,0.010104,0.021065,0.008112,0.066118


In [213]:
cov_stocks = matrix(np.array(cov_stocks))

In [214]:
cov_stocks

<4x4 matrix, tc='d'>

Each objective for quadratic  programming has 2 parts: <br>
a) Linear part: represented by cx, where c is vector of coefficients and x is vector of the decision variables <br>
b) Quadratic  part: represented by x.Q.x' <br>
<br>
Together the objective is minimize: cx - x.Q.x' <br><br>
So, our objective essentially is to minimize portfolio risk.<br>
We do this using a covariance matrix instead of just using the worst return entry for each ticker

In [215]:
objective = cov_stocks
objective

<4x4 matrix, tc='d'>

In our case, we do not have any linear objective, so we create a vector of coefficients as all zeros.

In [216]:
vector = matrix(np.zeros((n, 1)))

In [217]:
constraints_inequality_lhs = matrix(-np.transpose(np.array(expected_returns)))

In [218]:
constraints_inequality_lhs

<1x4 matrix, tc='d'>

In [232]:
min_expected_return = 0.10

In [233]:
constraints_inequality_rhs = matrix(-np.ones((1, 1)) * min_expected_return)

In [234]:
constraints_equality_lhs = matrix(1., (1, n))

In [235]:
constraints_equality_rhs = matrix(1.0)

In [236]:
solution = solvers.qp(objective, vector, constraints_inequality_lhs, constraints_inequality_rhs, constraints_equality_lhs, constraints_equality_rhs)

     pcost       dcost       gap    pres   dres
 0:  2.3416e-03  7.1796e-03  7e-02  1e+00  3e-18
 1:  2.5527e-03  7.7918e-03  8e-04  8e-02  2e-18
 2:  2.1409e-02  1.4137e-02  7e-03  2e-16  3e-17
 3:  1.5079e-02  1.4802e-02  3e-04  0e+00  3e-17
 4:  1.4806e-02  1.4803e-02  3e-06  1e-17  6e-18
 5:  1.4803e-02  1.4803e-02  3e-08  1e-17  1e-17
Optimal solution found.


In [237]:
weights_quad = solution["x"]
weights_quad

<4x1 matrix, tc='d'>

In [238]:
weights_quad = np.array(weights_quad)

In [239]:
weights_quad

array([[ 0.48353112],
       [-0.10793268],
       [ 0.00602701],
       [ 0.61837456]])

In [240]:
sum(weights_quad)

array([1.])

In [241]:
pf_risk_quad = np.dot(np.dot(weights_quad.T, cov_stocks), weights_quad)

In [242]:
pf_risk_quad

array([[0.0296064]])

In [243]:
pf_return_quad = np.dot(expected_returns.T, weights_quad)

In [244]:
pf_return_quad

array([[0.10000009]])